# Dynamic Programming

## Example 4.1

In [17]:
#######################################################################
# Copyright (C)                                                       #
# 2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

[ref](https://github.com/ShangtongZhang/reinforcement-learning-an-introduction/blob/master/chapter04/grid_world.py)

In [18]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.table import Table

In [19]:
# Use the 'Agg' backend for matplotlib, which is great for scripts generating files without displaying them.
matplotlib.use('Agg')

# Define the size of the gridworld.
WORLD_SIZE = 4
# Define possible actions in the gridworld: left, up, right, down.
ACTIONS = [np.array([0, -1]),  # left
           np.array([-1, 0]),  # up
           np.array([0, 1]),   # right
           np.array([1, 0])]   # down
# Probability of each action being taken. The agent follows the equiprobable random policy.
ACTION_PROB = 0.25

In [20]:
def is_terminal(state):
    """Check if the current state is a terminal state."""
    x, y = state
    # Terminal states are top-left and bottom-right corners of the grid.
    return (x == 0 and y == 0) or (x == WORLD_SIZE - 1 and y == WORLD_SIZE - 1)

In [21]:
def step(state, action):
    """Calculate the next state and reward given the current state and action."""
    if is_terminal(state):
        return state, 0  # No reward and remain in the same state if terminal.

    # Compute the next state based on current state and action.
    next_state = (np.array(state) + action).tolist()
    x, y = next_state

    # If next state is outside the grid, revert to the original state.
    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        next_state = state

    reward = -1  # Reward for each transition.
    return next_state, reward

In [22]:
def draw_image(image):
    """Create a plot to visualize the values of each state in the grid."""
    fig, ax = plt.subplots()
    ax.set_axis_off()
    tb = Table(ax, bbox=[0, 0, 1, 1])

    nrows, ncols = image.shape
    width, height = 1.0 / ncols, 1.0 / nrows

    # Populate the table with the state values.
    for (i, j), val in np.ndenumerate(image):
        tb.add_cell(i, j, width, height, text=val, loc='center', facecolor='white')

    # Add row and column labels.
    for i in range(len(image)):
        tb.add_cell(i, -1, width, height, text=i+1, loc='right', edgecolor='none', facecolor='none')
        tb.add_cell(-1, i, width, height/2, text=i+1, loc='center', edgecolor='none', facecolor='none')
    ax.add_table(tb)

In [23]:
def compute_state_value(in_place=True, discount=1.0):
    """Compute the state values using iterative policy evaluation."""
    new_state_values = np.zeros((WORLD_SIZE, WORLD_SIZE))  # Initialize state values to zero.
    iteration = 0
    while True:
        if in_place:
            state_values = new_state_values  # In-place updates.
        else:
            state_values = new_state_values.copy()  # Out-of-place updates.

        old_state_values = state_values.copy()

        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                value = 0
                for action in ACTIONS:
                    (next_i, next_j), reward = step([i, j], action)
                    value += ACTION_PROB * (reward + discount * state_values[next_i, next_j])
                new_state_values[i, j] = value

        max_delta_value = abs(old_state_values - new_state_values).max()
        if max_delta_value < 1e-4:
            break

        iteration += 1

    return new_state_values, iteration

In [24]:
def figure_4_1():
    """Reproduce Figure 4.1 from Sutton and Barto's book."""
    _, async_iteration = compute_state_value(in_place=True)
    values, sync_iteration = compute_state_value(in_place=False)
    draw_image(np.round(values, decimals=2))
    print('In-place: {} iterations'.format(async_iteration))
    print('Synchronous: {} iterations'.format(sync_iteration))

    plt.savefig('./images/figure_4_1.png')  # Save the figure to a file.
    plt.close()

In [25]:
figure_4_1()  # Run the function when the script is executed.

In-place: 113 iterations
Synchronous: 172 iterations


### The Bellman Equation for Policy Evaluation

In the provided Python code, the Bellman equation is applied within the `compute_state_value` function, specifically in the nested loop where the value of each state in the gridworld is calculated based on potential actions. Let’s break down how this section corresponds to the Bellman equation for policy evaluation.

The Bellman equation for policy evaluation in its simplest form states:

$$
V(s) = \sum_{a} \pi(a|s) \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V(s') \right]
$$

Where:
- $V(s)$ is the state-value function for state $s$,
- $\pi(a|s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$,
- $p(s', r | s, a)$ is the probability of transitioning to state $s'$ and receiving reward $r$ after taking action $a$ in state $s$,
- $r$ is the reward received after transitioning from $s$ to $s'$,
- $\gamma$ is the discount factor,
- $V(s')$ is the value of the next state.

### Application in the Code

The implementation in the provided code uses an equally likely policy, where each action is selected with equal probability ($\pi(a|s) = 0.25$ for each action). Here’s how the specific part of the code maps to the Bellman equation:

```python
for i in range(WORLD_SIZE):
    for j in range(WORLD_SIZE):
        value = 0
        for action in ACTIONS:
            (next_i, next_j), reward = step([i, j], action)
            value += ACTION_PROB * (reward + discount * state_values[next_i, next_j])
        new_state_values[i, j] = value
```

- **`value = 0`**: This initializes the sum for the current state’s value calculation.
- **`for action in ACTIONS`**: This loop over actions represents the sum over actions $a$ in the equation.
- **`(next_i, next_j), reward = step([i, j], action)`**: This function call calculates the next state and reward given the current state $[i, j]$ and action. It directly corresponds to $s'$ and $r$ in the equation.
- **`ACTION_PROB`**: This is $\pi(a|s)$, the probability of selecting each action which is 0.25, indicating an equal likelihood of choosing any action.
- **`(reward + discount * state_values[next_i, next_j])`**: This is the term $[r + \gamma V(s')]$ in the Bellman equation, where `reward` is $r$ and `state_values[next_i, next_j]` corresponds to $V(s')$.
- **`discount`**: This is the discount factor $\gamma$ in the equation, which scales the value of future rewards.
- **`new_state_values[i, j] = value`**: This assigns the computed value back to the state-value array for state $(i, j)$, effectively updating $V(s)$ for each state $s$ based on the Bellman equation.

The Bellman equation is used to iteratively update the value of each state based on the possible actions and the subsequent states and rewards. This iterative process is central to many reinforcement learning algorithms, especially those involving policy evaluation and improvement like in this example. The Bellman equation ensures that the value of each state properly accounts for the expected returns under the given policy, leading to the convergence of state values toward the true values as the algorithm iterates.

In the context of this simple example, which represents a typical gridworld scenario from reinforcement learning literature, $ p(s', r \mid s, a) = 1 $ for all states $s$ and actions $a$ because the environment is deterministic. In a deterministic environment, the outcome of each action taken in a state is completely predictable. There are no random elements affecting where you end up or what reward you get after taking an action. Therefore, when an action $a$ is taken in a state $s$, there is only one possible outcome state $s'$ and one possible reward $r$. Since each action from a state leads to a specific next state and specific reward deterministically, the transition probability is 1. There are no other possibilities to consider.

## A Simple Gridworld Example

Gridworld Setup:

- A $n \times n$ grid with states labeled from 0 to $n$.
- Actions possible are ***up, down, left***, and ***right***.
- Moving out of the grid keeps you in the same state.
- A reward of -1 on all transitions except those leading to the terminal state (state $n$), which has a reward of 0.

In [6]:
import numpy as np
#
# next_state Function: This computes the next state given a state and an action. 
# It handles edge cases to ensure the state stays within grid boundaries. 
#
def next_state(s, a, grid_size):
    """Determine the next state given action and current state."""
    if a == 'up':  # Move up unless at the top row
        return s if s < grid_size else s - grid_size
    elif a == 'down':  # Move down unless at the bottom row
        return s if s >= grid_size * (grid_size - 1) else s + grid_size
    elif a == 'left':  # Move left unless at the left edge
        return s if s % grid_size == 0 else s - 1
    elif a == 'right':  # Move right unless at the right edge
        return s if (s + 1) % grid_size == 0 else s + 1
#
# value_iteration Function: Implements value iteration. It iterates over each state, 
# updating the value function by considering all possible actions and choosing the one 
# that maximizes the state's value until the maximum change across all states between 
# iterations is less than a small threshold.
#
# Terminal State: In our setup, the lower right corner of the grid is terminal with a 
# reward of 0 and does not transition anywhere.
#
def value_iteration(grid_size, gamma=0.9, threshold=0.001):
    states = np.arange(grid_size**2)           # Array of all states from 0 to grid_size^2 - 1
    V = np.zeros(len(states))                  # Initialize value function array to zero
    delta = float('inf')                       # Large initial value for tracking convergence
    actions = ['up', 'down', 'left', 'right']  # Possible actions
    while delta > threshold:                   # Continue until change in value function is small
        delta = 0
        for s in states:
            if s == grid_size**2 - 1:  # Skip terminal state (assumed to be the last state)
                continue
            v = V[s]  # Current value of state s
            # Compute the maximum value of all actions from state s
            V[s] = max([sum([p * (r + gamma * V[next_state(s, a, grid_size)]) for p, r in [(1.0, -1)]])
                        for a in actions])
            # Update delta to be the maximum difference observed in this iteration
            delta = max(delta, abs(v - V[s]))
    return V  # Return the final value function

# Example usage:
grid_size = 4  # Change this to any grid size
optimal_values = value_iteration(grid_size)  # Run value iteration

print("Optimal values for each state in a {}x{} grid:".format(grid_size, grid_size))
for i in range(grid_size):
    print(' '.join("{:.2f}".format(val) for val in optimal_values[i*grid_size:(i+1)*grid_size]))

Optimal values for each state in a 4x4 grid:
-4.69 -4.10 -3.44 -2.71
-4.10 -3.44 -2.71 -1.90
-3.44 -2.71 -1.90 -1.00
-2.71 -1.90 -1.00 0.00


## Alternative Implementation (doesn't work!)

In [16]:
import numpy as np

def next_state(s, a, grid_size):
    """Determine the next state given action and current state."""
    if a == 'up':
        return s if s < grid_size else s - grid_size
    elif a == 'down':
        return s if s >= grid_size * (grid_size - 1) else s + grid_size
    elif a == 'left':
        return s if s % grid_size == 0 else s - 1
    elif a == 'right':
        return s if (s + 1) % grid_size == 0 else s + 1
    return s

def value_iteration(grid_size, gamma=1.0, threshold=0.01):
    states = np.arange(grid_size**2)  # Array of all states from 0 to grid_size^2 - 1
    V = np.zeros(len(states))  # Initialize value function array to zero
    delta = float('inf')  # Large initial value for tracking convergence
    actions = ['up', 'down', 'left', 'right']  # Possible actions
    terminal_states = {0, grid_size**2 - 1}  # Terminal states (upper-left and lower-right)

    while delta > threshold:  # Continue until change in value function is small
        delta = 0
        for s in states:
            if s in terminal_states:  # Skip terminal states
                continue
            v = V[s]  # Current value of state s
            # Compute the maximum value of all actions from state s
            V[s] = max([sum([p * (r + gamma * V[next_state(s, a, grid_size)]) for p, r in [(1.0, -1)]])
                        for a in actions])
            # Update delta to be the maximum difference observed in this iteration
            delta = max(delta, abs(v - V[s]))

    return V  # Return the final value function

# Example usage:
grid_size = 4  # 4x4 grid
optimal_values = value_iteration(grid_size, 0.99, 0.01)  # Run value iteration

print("Optimal values for each state in a {}x{} grid:".format(grid_size, grid_size))
for i in range(grid_size):
    print([f"{v:.2f}" for v in optimal_values[i*grid_size:(i+1)*grid_size]])  # Print formatted values


Optimal values for each state in a 4x4 grid:
['0.00', '-1.00', '-1.99', '-2.97']
['-1.00', '-1.99', '-2.97', '-1.99']
['-1.99', '-2.97', '-1.99', '-1.00']
['-2.97', '-1.99', '-1.00', '0.00']
